In [1]:
import math

In [ ]:
def first_diff_eq(H: float,#высота скважины ГС(определение при наклонной ГС?)(данные о скважине)
                  L: float,#длина ГС(данные о скважине)(обезразмеривание)
                  segments: list,#список с длинами сегметов для кислотной обработки(данные о скважине)
                  C_segments: list,#список с массовыми концетрациями кислоты для каждого сегмента(начальное условие)
                  q_w: float,#заданный расход на скважине(начальное условие)
                  r_w0: float,#начальный радиус скважины(начальное условие)
                  D_nkt: float,#внешний диаметр нкт(начальное условие)
                  t_star: float,#заднное в методичке время обезразмериванния(начальное условие)(обезразмеривание)
                  P_k: float,#давление задающееся для усреднения стр 21(начальное условие)
                  V: float,#reverse speed для нкт(начальное условие)
                  k: float,#проницаемость (пласта или трубы? изменяется с временем k(t)?)
                  m: float,#коэффециент пористости
                  P_ust: float,#устьевое давление скважины (задается как const?)
                  alpha: float,#угол отклонения ствола ГС от вертикали
                  p: float,#плотность кислоты или водяного раствора закачиваемого в скважину
                  g: float,#ускорение свободного падения
                  mu: float, #динамическая вязкость жидкости,
                  d_nkt: float, #внутренний диаметр нкт
                  xi: float, #коэффициент пъезопроводности продуктивного пласта 𝑘/𝜇(𝑚𝛽 + 𝛽𝑐)
                  ) -> list:
    list_of_greed = []
    list_of_t = []
    u_star = ((4 * q_w) / (math.pi * pow(D_nkt,2)))#начальное условие для скорости
    t_star = L / u_star
    P_star = P_k #начальное условие для давления

    u_grid = [[0] * list_of_greed.__len__() for i in range(list_of_t.__len__())]#сетка для скорости жидкости
    P_grid = [[0] * list_of_greed.__len__() for i in range(list_of_t.__len__())]#сетка для давления
    ksi = [0] * list_of_t.__len__()#сетка для велечины подвижной границы
    r_w = [0] * list_of_t.__len__()#сетка для радиуса скважины
    v_L = [[0] * list_of_greed.__len__() for i in range(list_of_t.__len__())]#сетка для скорости утечек жидкости через боковую поверхность ГС
    R_grid = [[0] * list_of_greed.__len__() for i in range(list_of_t.__len__())]#сетка для R
    C_grid = [[0] * list_of_greed.__len__() for i in range(list_of_t.__len__())]#сетка для массовыми концетрациями кислоты для каждого сегмента

    ksi[0] = 1
    r_w[0] = r_w0

    for i in range(list_of_greed.__len__()):
        u_grid[0][i] = 0 
        P_grid[0][i] = 1
        R_grid[0][i] = r_w0

    P_grid[0][0] = (P_ust + p * g * H - ((renaults_coeff(q_w,p,2 * r_w[0],mu) * 8 * H * p * pow(q_w,2)) / (pow(math.pi,2) * pow(d_nkt,5)))) / P_star
    for i in range(list_of_t.__len__()):
        u_grid[i][0] = 1 #проблема накладывание нач и гран условий!!
        u_grid[i][-1] = 0
        # P_grid[i][0] = (P_ust + p * g * H - ((renaults_coeff(q_w,p,2 * r_w[i],mu) * 8 * H * p * pow(q_w,2)) / (pow(math.pi,2) * pow(d_nkt,5)))) / P_star
    for i in range(list_of_greed.__len__()):
        v_L[0][i] = (k / (m * mu)) * (P_star / (u_star * L)) * (P_grid[0][i] - 1) / (R_grid[0][i] - r_w[0] + r_w[0] * math.log(math.sqrt(pow(r_w[0],2) + ((4 * xi) / pow(L,2)) * t_star * list_of_t[0]) / R_grid[0][i]))

In [ ]:
def renaults_coeff(Q: float,#расход жидкости
                   p: float,#плотность жидкости
                   D_w: float,#внутренний диаметр трубопровода
                   mu: float) -> float: #динамическая вязкость жидкости 
    ok = (4 * Q * p) / (math.pi * D_w * mu) 
    if ok < 2300:
        return (64. / ok)
    elif ok > 2800 and ok < 1e5:
        return (0.3164 / pow(ok,1./4))
    else:
        ??????????????????????